In [14]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import h5py
import os
import seaborn as sns


In [15]:
from scf_runs import run_casci, run_lda_be
scf_checkfile, opt_checkfile, mf_lda = run_lda_be()
ci_checkfile, mc = run_casci(scf_checkfile, nroots=1, ncas =3, nelecas = (2,2))

Be atom neutral LDA spin=0
HF
converged SCF energy = -14.5729681271986  <S^2> = 6.6613381e-16  2S+1 = 1
LDA
converged SCF energy = -14.4465058449002  <S^2> = 3.7747583e-15  2S+1 = 1
CASCI nelecas up/down (2, 2)
CASCI E = -14.5838197989191  E(CI) = -14.5838197989191  S^2 = 0.0000000
Available output from CASCI: <KeysViewHDF5 ['ci', 'fci', 'mo_coeff', 'ncas', 'nelecas']>


In [16]:
import bosonrecipes
import importlib, bosonrecipes, linemin, bosonwftools, mc, bosonslater, bosonaccumulators, stochastic_reconfiguration
modules = [importlib, bosonrecipes, linemin, bosonwftools, mc, bosonslater, bosonaccumulators, stochastic_reconfiguration]
for m in modules:
    importlib.reload(m)


In [17]:
# abvmcopt_file = 'abvmc-j-opt-be.hdf5'
# for fname in [abvmcopt_file]:
#     if os.path.isfile(fname):
#         os.remove(fname)
# num_it = 40
# bosonrecipes.ABOPTIMIZE(scf_checkfile, 
#                         abvmcopt_file, 
#                         ci_checkfile   = ci_checkfile,
#                         max_iterations = num_it, 
#                         jastrow_kws    = {"ion_cusp":False, 'na':0},
#                         verbose        = True,                            
#                         nconfig        = 1000)  


In [18]:
import concurrent
abvmcopt_file = 'abvmc-j-opt-be.hdf5'
num_it = 20
ncore = 16 
for fname in [abvmcopt_file]:
    if os.path.isfile(fname):
        os.remove(fname)
with concurrent.futures.ProcessPoolExecutor(max_workers=ncore) as client:
    bosonrecipes.ABOPTIMIZE(scf_checkfile, 
                    abvmcopt_file, 
                    ci_checkfile = ci_checkfile,
                    max_iterations=num_it, 
                    verbose=True,  
                    load_parameters = None, 
                    jastrow_kws={"ion_cusp":False, 'na':0},
                    client = client, 
                    npartitions=ncore,                            
                    nconfig=1000)


Using spherical guess
starting warmup
finished warmup
----------vmc done
Gradient norm:  1.2060380693199801
Dot product between gradient and SR step:  0.78893886385957
[ 0.09312619  0.09462315  0.25013974  0.25111786  0.3212172   0.32165813
  0.23594928  0.23767848  0.0048993  -0.03319157  0.00882985 -0.00672263
 -0.08249049 -0.00388047 -0.0063255  -0.12036757 -0.00543988]
descent en -13.419349709194615 0.005904207208597367
energies from correlated sampling [-12.53448944 -12.8649407  -13.18286268 -13.45168386 -13.6239459 ]
----------vmc done
Gradient norm:  0.647847805443809
Dot product between gradient and SR step:  0.8580071564625995


Process SpawnProcess-25:
Traceback (most recent call last):
  File "/Users/ksu/Software/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/ksu/Software/anaconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ksu/Software/anaconda3/lib/python3.11/concurrent/futures/process.py", line 249, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ksu/Software/anaconda3/lib/python3.11/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
          ^^^^^^^^^^^^^^^^^^
  File "/Users/ksu/Software/anaconda3/lib/python3.11/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ksu/Software/anaconda3/lib/python3.11/multiprocessing/connection.py", line 430, in _recv_bytes
    buf = self._recv(4)

KeyboardInterrupt: 

In [ ]:
import pyqmc.api as pyq
df = pyq.read_opt(abvmcopt_file)
df

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df['energy'], '-ok')